В скрипте нужно указать:  
* Путь к папке, где лежат скриншоты экрана kik; там должны быть:  
 * подпапки queue и ready
 * файл hashtags.csv
* Путь к исполняемому файлу Тессеракт


Что происходит:  
* Скрипт открывает по 30 файлов из папки queue
* Преобразует файлы (обрезает изображение и поляризует)
* Распознает текст на изображении

* Сохраняет преобразованный файл в папке ready
* Удаляет файл из папки queue

* Пишет в файл hashtags.csv
 * Хэштэг
 * Имя группы
 * Дату записи
 * Имя файла-источника

**Процесс установки:**

In [1]:
#Сначала установим библиотеки:
#!pip install PIL
#!pip install pytesseract
#Установить тесеракт и прописать путь к нему в path Windows
## Исходник тут: https://github.com/tesseract-ocr/tesseract/wiki/4.0-with-LSTM#400-alpha-for-windows
## В \Tesseract-OCR\tessdata выложить файл rob.traineddata , обученный распознавать Roboto (шрифт кика)

**Имя папки для работы со скриншотами:**

In [2]:
env_path = 'C:\\HOME\\KWPDS\\kik_env\\screenshots\\'

In [3]:
from PIL import Image, ImageDraw
import pytesseract
import datetime
import csv
import os

In [4]:
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe'

In [5]:
screenshots_queue = os.listdir(path=env_path+'queue')[0:30]
on_recognize_queue = []
dt = datetime.datetime.utcnow().strftime('%Y-%m-%d %I:%M%p')

In [6]:
for shot in screenshots_queue:
    #Crop the image
    in_file_name = env_path+'queue\\'+shot
    img = Image.open(in_file_name)
    w, h = img.size
    img = img.crop((110, 0, w-130, h))
    w, h = img.size
    
    # Set levels
    img_draw = ImageDraw.Draw(img)
    pix = img.load()
    
    for i in range(w):
        for j in range(h):
            a = pix[i, j][0]
            b = pix[i, j][1]
            c = pix[i, j][2]
            S = a + b + c

            if S < 625:
                a,b,c = map(lambda x: ((x-100)//3)*2, [a, b, c])
            else:
                a, b, c = 255, 255, 255

            img_draw.point((i, j), (a, b, c))
    
    out_file_name = env_path+'ready\\'+shot
    img.save(out_file_name, "PNG")
    on_recognize_queue.append(out_file_name)
    
    os.remove(in_file_name)
    del img_draw

In [7]:
text_list = []
hashtag_list = []

for file_name in on_recognize_queue:
    img_from = Image.open(file_name)                  
    text = pytesseract.image_to_string(img_from, lang='rob', config ='tessedit_char_whitelist = #')
    text_list.append(text)

    for t in text_list:
        #Cut incorrect data
        t = [i for i in t.split('\n') if len(i) >1]
        if 'Groups' == t[4]:
            t = t[2:4]+t[5::]
        while '#' not in t[-1]:
            t = t[:len(t)-1:]

        #Convert data to hashtag list
        gap = 0
        for i in range(1, len(t),2):
            if i+gap < len(t):
                item = t[i+gap]
                while '#' not in item:
                    gap += 1
                    item = t[i+gap]
                hashtag_list.append([t[i+gap-1], item, dt, file_name])            

In [8]:
output_file = env_path+'hashtags.csv'

with open(output_file, 'a') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';',quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    for r in hashtag_list:
        spamwriter.writerow(r)

csvfile.close()